## 6. Transfer learning

Source: https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/

Transfer learning is the process of utilising knowledge from a particular task/domain to model for another task/domain.  

cap10

One approach of transfer learning is the use of pre-trained model. In this capstone project, we will utilise Google's mobilenet ConvNet model and retrain the last layer of the model to classify our 17 groceries. 

### 6.1 Retraining the pre-trained model
Retraining the model is as simple as loading up the model in your terminal, and specifying the images that you wish to classify.

In [ ]:
python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=500 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/"${ARCHITECTURE}" \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture="${ARCHITECTURE}" \
  --image_dir=tf_files/grocery-photos

Next, we would define 2 functions to classify our image. We would define the first function, load_graph, to load our rerained model. The second function, read_tensor_from_image_file, will return the value of the class based on the model specified.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse

import numpy as np
import tensorflow as tf


def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph


def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
        image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)

    return result


def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

### 6.2 Load the graph (model) and labels for classification

In [2]:
graph = load_graph('../tensorflow-for-poets-2/tf_files/retrained_graph.pb')
labels = load_labels('../tensorflow-for-poets-2/tf_files/retrained_labels.txt')

In [3]:
input_height=128
input_width=128
input_mean=0
input_std=255

In [4]:
input_layer = "input"
output_layer = "final_result"

### 7.1 Classifying image #1, class 0

In [5]:
# Build confusion matrix
import os
PATH = os.getcwd()
print(PATH)

/home/Jeryl/tensorflow-for-poets-2


In [6]:
image_location = os.path.join(PATH,'tf_files','conf')
data_dir_list = os.listdir(image_location)

In [7]:
y_pred = []
for dataset in data_dir_list:
    img_folder=os.path.join(image_location,dataset)
    img_dir = os.listdir(img_folder)
    for img in img_dir:
        file_name= os.path.join(image_location,dataset,img)
        t = read_tensor_from_image_file(
              file_name,
              input_height=input_height,
              input_width=input_width,
              input_mean=input_mean,
              input_std=input_std)

        input_name = "import/" + input_layer
        output_name = "import/" + output_layer
        input_operation = graph.get_operation_by_name(input_name)
        output_operation = graph.get_operation_by_name(output_name)

        with tf.Session(graph=graph) as sess:
            results = sess.run(output_operation.outputs[0], {
                input_operation.outputs[0]: t
            })
            results = np.squeeze(results)

        top_k = results.argsort()[-5:][::-1]
        print('read image in folder: ', dataset, img)
        y_pred.append(labels[top_k[0]])

read image in folder:  8 _0_423.jpg
read image in folder:  8 _0_602.jpg
read image in folder:  8 web1.jpg
read image in folder:  8 web3.jpg
read image in folder:  8 web2.jpg
read image in folder:  4 snick3.jpg
read image in folder:  4 snick4.jpg
read image in folder:  4 snick5.jpg
read image in folder:  4 snick1.jpg
read image in folder:  4 snick2.jpeg
read image in folder:  11 _0_705.jpg
read image in folder:  11 _0_2735.jpg
read image in folder:  11 _0_2633.jpg
read image in folder:  11 web1.jpg
read image in folder:  11 _0_2833.jpg
read image in folder:  6 web1.jpg
read image in folder:  6 web4.jpg
read image in folder:  12 _0_125.jpg
read image in folder:  12 _0_83.jpg
read image in folder:  12 web1.jpg
read image in folder:  12 _0_1551.jpg
read image in folder:  12 web2.jpg
read image in folder:  15 web5.jpg
read image in folder:  15 _0_1843.jpg
read image in folder:  15 _0_1176.jpg
read image in folder:  15 _0_1219.jpg
read image in folder:  15 web3.jpg
read image in folder:  0 w

In [11]:
Y_pred = []
for i in y_pred:
    Y_pred.append(int(i))
print(Y_pred)

[8, 8, 8, 11, 11, 13, 13, 14, 7, 9, 11, 11, 11, 11, 11, 13, 13, 12, 12, 12, 12, 12, 11, 15, 15, 15, 11, 11, 11, 7, 11, 7, 10, 10, 10, 10, 10, 5, 5, 5, 5, 5, 13, 7, 14, 14, 10, 14, 14, 14, 14, 14, 11, 14, 11, 7, 7, 14, 13, 13, 13, 13, 13, 16, 16, 11, 16, 16, 11, 14, 14, 11, 14, 11, 11, 14, 14, 5, 11, 14, 12, 4, 4, 11, 12, 14]


In [12]:
import pickle

Y_pred = np.array(Y_pred)
Y_pred = Y_pred.astype('int8')

# Pickle y_train
pickle_out = open('trfconf','wb')
pickle.dump(Y_pred, pickle_out)
pickle_out.close()

del Y_pred

In [21]:
# getting true class
conftrue = []
for dataset in data_dir_list:
    img_folder=os.path.join(image_location,dataset)
    img_dir = os.listdir(img_folder)
    for img in img_dir:
        conftrue.append(dataset)
print(conftrue)

['8', '8', '8', '8', '8', '4', '4', '4', '4', '4', '11', '11', '11', '11', '11', '6', '6', '12', '12', '12', '12', '12', '15', '15', '15', '15', '15', '0', '0', '0', '0', '0', '10', '10', '10', '10', '10', '5', '5', '5', '5', '5', '3', '3', '3', '3', '3', '14', '14', '14', '14', '14', '7', '7', '7', '7', '7', '7', '13', '13', '13', '13', '13', '16', '16', '16', '16', '16', '9', '9', '9', '9', '9', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2']


In [22]:
conftrue2 = []
for i in conftrue:
    conftrue2.append(int(i))

conftrue2 = np.array(conftrue2)
conftrue2 = conftrue2.astype('int8')

In [23]:
# Pickle y_train
pickle_out = open('conftrue','wb')
pickle.dump(conftrue2, pickle_out)
pickle_out.close()

In [34]:
file_name= '../tensorflow-for-poets-2/tf_files/augment/2/_0_1146.jpg'
t = read_tensor_from_image_file(
      file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
    results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]

print('Predicted class for your image is: ', labels[top_k[0]])

Predicted class for your image is:  2


In [27]:
top_k

array([ 4,  3, 12, 15,  8])

### 7.2 Classifying image #2, class 5

In [21]:
file_name= '../tensorflow-for-poets-2/tf_files/augment/class5.jpg'
t = read_tensor_from_image_file(
      file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
    results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]

results

NotFoundError: ../tensorflow-for-poets-2/tf_files/augment/class5.jpg; No such file or directory
	 [[Node: file_reader_15 = ReadFile[_device="/job:localhost/replica:0/task:0/device:CPU:0"](file_reader_15/filename)]]

Caused by op 'file_reader_15', defined at:
  File "/home/Jeryl/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/Jeryl/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/Jeryl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/Jeryl/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/Jeryl/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/Jeryl/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(

### 7.3 Classifying image #3, class 3

In [24]:
file_name= '../tensorflow-for-poets-2/tf_files/augment/class3.jpg'
t = read_tensor_from_image_file(
      file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
    results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]

for i in top_k:
    print(labels[i], results[i])

10 0.450499
11 0.270105
13 0.13487
15 0.0740191
16 0.0211695


## Summary
Image classification using deep learning can have significant impacts on society. In order to build a successful image classifier, we would need to establish a robust model for classification. The robustness of the model can be tweaked by the various hyper-parameters in the particular model of choice.  

In this project, we have explored using the vanilla ConvNet architecture as well as a pre-trained ConvNet architecture made available by Google through transfer learning. In the vanilla ConvNet architecture, we have seen how we could tweak the architecture based on our preferences. On the other hand, we have also observed from transfer learning, the ease at which businesses can employ pre-existing models to help with their image classification problems.  

In my personal opinion, I would advise businesses to take a look at their dataset before considering to build their own model for classification. Businesses can then pick and choose their best model based on their needs. Here are some advantages of both methods:  
  
  
**1. Build ConvNet from scratch**
- Full control of hyper-parameters
- Full control of trainable features, depth and complexity of the model  

**2.  Transfer learning **
- Easy to apply
- Models from different domains may be applicable to business use cases from differing domains

## Future work
1. Explore other techniques to classify images: Support Vector Machines, Fuzzy measures and Genetic Algorithms
2. Build application for image classification